<a href="https://colab.research.google.com/github/Jonchyk/Datamgmt/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

!pip install pyreadstat

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.6 MB/s eta 0:00:00


In [8]:
#read spss file
df = pd.read_spss("fs.sav")
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns', None)  # No limit on columns
#let's take a peek at this data set.
df.head(10)
#Let's specifically look at our geographic breakdown, since this is important
df["stratum"].value_counts()

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRE

,count
stratum,
Naryn - rural,480
Talas - rural,443
Osh city - urban,430
Osh - rural,387
Chui - rural,360
Batken - rural,327
Bishkek city - urban,308
Jala-Abad - rural,296
Issyk-kul - rural,284


In [12]:
#let's get this down to 8 to 10 years old, since the data collected is for ages 7-14, but is a G2 foundational test. Wow!
df_age = df[(df["CB3"] > 7) & (df["CB3"] < 11)]
#how many respondents do we have now?
df_age["stratum"].value_counts()

,count
stratum,
Osh city - urban,108
Talas - rural,106
Naryn - rural,105
Osh - rural,103
Chui - rural,97
Batken - rural,76
Bishkek city - urban,73
Issyk-kul - rural,73
Jala-Abad - rural,72


In [15]:
#further, let's see how many of these children consented to taking hte test.
df_age["FL3"].value_counts()

,count
FL3,
YES,897
NO / NOT ASKED,33


In [18]:
#897 said yes! Of those 897, how many do we have in each "stratum" now?
df_age[["stratum", "FL3"]].value_counts(normalize=False)
#ok! This looks like it's enough data to work with.  While some of the urban smaples are a bit low, every sample per region is >30!


,,count
stratum,FL3,
Naryn - rural,YES,103
Talas - rural,YES,102
Osh city - urban,YES,98
Chui - rural,YES,94
Osh - rural,YES,85
Issyk-kul - rural,YES,73
Batken - rural,YES,72
Bishkek city - urban,YES,61
Jala-Abad - rural,YES,61
